# TP10 : Matrices de confusion

Dans le TP précédent, nous avons mis en œuvre une partie de l'algorithme d'apprentissage supervisé des $k$-plus proches voisins vu en cours, dans le but de classer des images contenant des chiffres. Nous avions fixé une valeur de $k$.

Nous continuons à travailler sur la même base d'images, mais le but cette semaine est de comparer l'efficacité de plusieurs valeurs de $k$ pour en trouver une bonne.

Pour rappel, l'idée est de fabriquer une matrice de confusion qui donne le nombre d'images mal classifiées.

Les algorithmes mis en place la semaine dernières étaient assez lents, il n'est pas question de classifier les 10000 images de test avec. Nous allons tenter de les améliorer.

Je reprends le code de la semaine dernière pour mettre les images en mémoire (pour rappel a été largement emprunté au site [https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook](https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook), et légèrement modifié).

In [ ]:
import numpy as np # linear algebra
import struct
from array import array
import gzip

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with gzip.open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with gzip.open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array('B', file.read())
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img.tolist()
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)

Le code qui suit utilise la classe précédente pour récupérer les images (quelques secondes d'exécution, soyez patients).

In [ ]:
training_images_filepath = 'train-images-idx3-ubyte.gz'
training_labels_filepath = 'train-labels-idx1-ubyte.gz'
test_images_filepath = 't10k-images-idx3-ubyte.gz'
test_labels_filepath = 't10k-labels-idx1-ubyte.gz'

#
# Load MINST dataset
#
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath,
                                   test_images_filepath, test_labels_filepath)
(img_entrainement, cl_entrainement), (img_test, cl_test) = mnist_dataloader.load_data()

On va tester certaines fonctions avec des affichages, on reprend donc la fonction d'affichage du TP précédent:

In [ ]:
from PIL import Image
from IPython.display import display

def affiche_image(img):
    """img : image sous forme d'une liste de listes de niveaux de gris
    affichage de l'image"""
    im = Image.new(size=(len(img[0]), len(img)), mode='L')
    for i in range(len(img)):
        for j in range(len(img[0])):
            im.putpixel((j, i), img[i][j])
    display(im)

On aura besoin de l'algorithme d'insertion écrit dans le TP précédent:

In [ ]:
def inserer(elt, liste):
    """elt: couple (image, distance) à insérer dans liste
    pré-condition : la distance de elt est < à la dist de liste[-1] 
    le résultat doit être décroissant sur les distances et conserver sa longueur"""
    img, d = elt
    assert(d < liste[-1][1])
    i = len(liste) - 1
    liste[i] = (img, d)
    while i > 0 and liste[i][1] < liste[i-1][1]:
        liste[i-1], liste[i] = liste[i], liste[i-1]
        i = i-1

### Exercice 1 : Transformer les images pour accélérer les calculs
Tant que $k$ est petit, l'étape qui prend beaucoup de temps est le calcul de la distance euclidienne entre deux images. On va pré-traiter les images pour essayer d'accélérer les choses.

Première étape: on enlève les niveaux de gris et on ne garde que des pixels noirs et des pixels blancs.
Écrire une fonction `binaire` qui prend en argument une image `img` sous la forme d'une liste de listes de niveau de gris et un entier `seuil` et rennvoie une image aux mêmes dimensions dont la coordonnée $(i, j)$ vaut 0 si la valeur de la case $(i, j)$ de `img` est strictement inférieure à `seuil` et 255 sinon.

Comparer les deux affichages suivants pour savoir si votre fonction est correcte:

In [ ]:
affiche_image(img_entrainement[257])
affiche_image(binaire(img_entrainement[257], 100))

Créez maintenant, à partir des listes `img_entrainement` et `img_test` deux listes d'images correspondantes avec des pixels tous noirs ou blancs:

Petite vérification :

In [ ]:
affiche_image(img_entrainement[257])
affiche_image(img_bin_entrainement[257])

### Exercice 2 : Accélérer le calcul de la distance entre deux images
Pour diminuer le nombre d'opérations de ce calcul, nous allons utiliser deux idées:

1. Si on sait déjà qu'on cherche une distance inférieure à une valeur donnée (par exemple la distance au $k$-ème plus proche voisin pour l'instant), ce n'est pas la peine de continuer le calcul si cette valeur est atteinte,
2. Les traits étant continus, on ne va regarder qu'une ligne sur 5 dans les images.

Écrire une fonction `distance_images` qui prend en argument deux images données sous la forme de listes de listes de noirs et de blanc et une nombre `d` et renvoie le nombres de pixels différents entre les deux images s'il est inférieur à `d` et `math.inf` sinon, en ne regardant qu'une ligne sur 5 et en essayant de faire le moins de calculs possibles.

### Rappel TP précédent : trouver les $k$ plus proches voisins

Je remets les fonctions que vous deviez écrire dans le TP précédent, en incorporant la modification dûe au fait qu'on ne calcule pas la distance dès qu'on est convaincu que l'élément ne fait pas partie des $k$ plus proches voisins.

In [ ]:
def k_plus_proches_voisins(im, L, k, dist, insertion):
    """im: image (liste de listes de noir ou blanc)
    L: liste d'images (noir ou blanc)
    k: entier (k < len(L))
    dist: fonction de calcul de distance
    insertion: fonction d'insertion"""
    voisins = [(None,math.inf)] * k
    for i in range(len(L)):
        orig = L[i]
        d = dist(im, orig, voisins[-1][1])
        if d < voisins[-1][1]:
            insertion((i, d), voisins)
    return voisins

In [ ]:
def classe_devinee(voisins, classes):
    """liste de voisins (numero, distance) et liste de classes"""
    classes_voisins = [ classes[n] for (n,_) in voisins ]
    cpt = {i:0 for i in range(10)}
    for cl in classes_voisins:
        cpt[cl] += 1
    classe = 0
    for cl in cpt:
        if cpt[cl] > cpt[classe]:
            classe = cl
    return classe

### Exercice 3 : Calcul de la matrice de confusion
Il s'agit maintenant d'ćrire une fonction qui calcule la matrice de confusion pour un entier $k$ donné.

Le python n'étant pas un langage qui s'exécute très rapidement, nous allons nous contenter de faire des tests avec `img_bin_test[:100]`.

Écrire une fonction qui prend en argument un entier $k$ et produit la matrice de confusion pour le calcul des classes avec les $k$ plus proches voisins des éléments de `img_bin_test[:200]`.

In [ ]:
matrice5 = matrice_de_confusion(5)

In [ ]:
assert(matrice5[0][0] == 17)
assert(matrice5[5][3] == 3)

### Exercice 4 : Calcul du taux d'échec
Écrire une fonction `taux_echec` qui prend en argument une matrice supposée être une matrice de confusion et renvoie son taux d'échec.

Comparez les taux d'échec pour $k=3$, $k=6$ et $k=20$. (Attention: on a réduit les temps de calcul, mais ça reste très long car on ne dispose pas des bons outils pour ce qu'on fait...)

In [ ]:
matrice3 = matrice_de_confusion(3)

In [ ]:
taux_echec(matrice3)

In [ ]:
matrice3

In [ ]:
matrice6 = matrice_de_confusion(6)

In [ ]:
taux_echec(matrice6)

In [ ]:
matrice6

In [ ]:
matrice20 = matrice_de_confusion(20)

In [ ]:
taux_echec(matrice20)

In [ ]:
matrice20